# LangGraph Agent v1 — Basic Q&A

Single-node agent that answers directly with the LLM (no tools).

Quick guide:
- State: messages
- Node: analyze (answer_question) → END
- Observe prints: [ANSWER]
- Run the last cell to see a clean, single execution trace

In [51]:
import os
from typing import List, Optional, Union, Literal
from dotenv import load_dotenv
from pydantic import BaseModel, Field

# LangGraph
from langgraph.graph import StateGraph, END

# LangChain core
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

True

In [52]:
class SearchResult(BaseModel):
    id: int
    title: str
    snippet: str
    link: str
    relevance_score: Optional[float] = None
    full_content: Optional[str] = None

class AgentState(BaseModel):
    messages: List[Union[HumanMessage, AIMessage, SystemMessage]] = Field(description="The chat history")
    class Config:
        arbitrary_types_allowed = True

In [53]:
# Initialize LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.7)

In [54]:
def answer_question(state: AgentState):
    print("[ANSWER] Executing answer_question node…")
    user_query = next((m.content for m in reversed(state.messages) if isinstance(m, HumanMessage)), "")
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a knowledgeable assistant. Be concise and factual."),
        ("human", "Answer: {q}")
    ])
    ans = llm.invoke(prompt.format_messages(q=user_query)).content.strip()
    return {"messages": [AIMessage(content=ans)]}

In [55]:
def is_done(state: AgentState) -> Literal["end"]:
    return "end"

In [56]:
# Build graph
workflow_v1 = StateGraph(AgentState)
workflow_v1.add_node("analyze", answer_question)
workflow_v1.add_conditional_edges("analyze", is_done, {"end": END})
workflow_v1.set_entry_point("analyze")
agent = workflow_v1.compile()

In [ ]:
# Test v1
query = "What teams were in NBA playoffs in 2025?"
initial_state = AgentState(messages=[HumanMessage(content=query)])
final_state = agent.invoke(initial_state)


[ANSWER] Executing answer_question node…
==== Execution Trace ====
(Watch for: [ANSWER])

==== Conversation ====
AI: As of October 2024, the teams that will be in the 2025 NBA playoffs are unknown. The playoffs occur at the end of the NBA season, and team qualification depends on their performance during the regular season.
==== Execution Trace ====
(Watch for: [ANSWER])

==== Conversation ====
AI: As of October 2024, the teams that will be in the 2025 NBA playoffs are unknown. The playoffs occur at the end of the NBA season, and team qualification depends on their performance during the regular season.


In [ ]:
for m in final_state['messages']:
    if isinstance(m, HumanMessage):
        print(f"Human: {m.content}")
    elif isinstance(m, AIMessage):
        print(f"AI: {m.content}")
    else:
        print(m)